# N-gram and Vectorization
* part 1. Build N-gram based Tokenizer
<br> - unigram and bigram
<br> - N-gram and Tokenization


* part 2. DTM and Vectorization
<br> - term frequency matrix
<br> - text classification with Logistic Regression

<br> <참고> 
<br> https://lovit.github.io/nlp/2018/10/23/ngram/
<br> https://soohee410.github.io/compare_tagger
<br> https://lovit.github.io/nlp/2018/03/26/from_text_to_matrix/

In [2]:
import pandas as pd
import numpy as np

import os

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [9]:
# check directory
os.listdir('./data/')

['3_class_naver_news_200819_test.csv',
 '3_class_naver_news_200819_train.csv',
 '2016_filtered_review.txt']

In [96]:
train = pd.read_csv('./data/3_class_naver_news_200819_train.csv', header=None)
test = pd.read_csv('./data/3_class_naver_news_200819_test.csv', header=None)
train.head(3)

,0,1,2,3,4,5,6
0,0,20170108,IT_science,데일리안,‘효자폰’ 갤럭시S7 ‘불효폰’ G5…분위기 극과극,삼성전자 ‘갤럭시S7’ 왼쪽 LG전자 ‘G5’ 오른쪽 모델 이미지 ⓒ각 사 삼성전자...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,1,20170111,IT_science,디지털타임스,카카오 O2O 플랫폼 사업 윤곽도 못잡아,11월초 공개 O2O 위드 카카오 두달 넘게 모습 드러내지 않아 후발주자로 출시까지...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,2,20170103,economy,조세일보,동원 고위도高緯度 북대서양 참다랑어 어획 성공,동원산업 대표이사 이명우 은 고위도 高緯度 북대서양 참다랑어를 어획하는데 성공했다고...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [84]:
print(train.shape)
print()
print(train.info())

(10000, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       10000 non-null  int64 
 1   1       10000 non-null  int64 
 2   2       10000 non-null  object
 3   3       10000 non-null  object
 4   4       10000 non-null  object
 5   5       10000 non-null  object
 6   6       10000 non-null  object
dtypes: int64(2), object(5)
memory usage: 547.0+ KB
None


In [97]:
colnames = ['index', 'date', 'category', 'media', 'title', 'contents', 'link']
train.columns = colnames
test.columns = colnames

## MeCab-based tokenization example

In [86]:
# tokenization with Mecab
from konlpy.tag import Mecab
mecab = Mecab()
print(mecab.morphs(train['title'][0]))

['‘', '효자', '폰', '’', '갤럭시', 'S', '7', '‘', '불효', '폰', '’', 'G', '5', '…', '분위기', '극', '과', '극']


## 1. N-gram extraction to build pre-trained dict
<br> 10,000개의 train data를 활용해서 n-gram dict를 만듭니다

In [106]:
from collections import defaultdict

def to_ngrams(words, n):
    ngrams = []
    for idx in range(len(words) - n + 1):
        ngrams.append(tuple(words[idx:idx+n]))
    return ngrams

def ngram_extractor(docs:list, min_count=5, n_range=(1,2)):
    
    '''
        input
        docs -> list whose elements are strings
        min_count -> minimum threshold for dictionary, default 5.
        n_range -> range of n-gram, default is for unigram and bigram.
        
        output
        ngram_counter -> counting dictionary for n-gram
    '''
    
    ngram_counter = defaultdict(int) # count를 위한 dictionary를 만들기 위해 defaultdict 사용
    begin, end = n_range
    
    for idx in range(len(docs)):
        
        if idx % 1000 == 0:
            print(f'processing {idx}th document...')
        
        unigrams = mecab.pos(docs[idx], join=True)
        for n in range(begin, end + 1):
            for ngram in to_ngrams(unigrams, n):
                ngram_counter[ngram] += 1

    # set min_count for the size of dict
    ngram_counter = {
        ngram:count for ngram, count in ngram_counter.items()
        if count >= min_count
    }
    
    return ngram_counter

ngram_dic = ngram_extractor(train['contents'])

processing 0th document...
processing 1000th document...
processing 2000th document...
processing 3000th document...
processing 4000th document...
processing 5000th document...
processing 6000th document...
processing 7000th document...
processing 8000th document...
processing 9000th document...


In [ ]:
# check built vocabs
ngram_dic

## 2. Use N-gram tokenizer
<br> tokenize documents using pre-trained Ngram tokenizer

In [107]:
from collections import defaultdict

class Ngram_tokenizer_MeCab():
    '''
        input
        vocab_dic -> pretrained n-gram vocab dictionary
        n_range -> range for n in n-gram
    '''
    def __init__(self, vocab_dic, n_range=(1,2)):
        self.vocab_dic = vocab_dic
        self.begin, self.end = n_range
        
    def __call__(self, sent):
        return self.tokenize(sent)

    def tokenize(self, sent):
        '''
            input
            sent -> a string or document, which is to be tokenized
            
            output
            ngrams -> tokenized result for given string or document
        '''
        if not sent:
            return []
        
        unigrams = mecab.pos(sent, join=True)
        
        ngrams = []
        for n in range(self.begin, self.end + 1):
            for ngram in self._to_ngrams(unigrams, n):
                ngrams.append('-'.join(ngram)) # to make it clear
        return ngrams

    def _to_ngrams(self, tokens, n): # returns n-gram for given window size n
        ngrams = []
        for idx in range(len(tokens) - n + 1):
            ngram = tuple(tokens[idx:idx+n])
            if ngram in self.vocab_dic:
                ngrams.append(ngram)
        return ngrams

In [108]:
tokenizer = Ngram_tokenizer_MeCab(vocab_dic = ngram_dic)

In [109]:
# test case for unigram and bigram
tokenizer.tokenize('나는 학교에 갔습니다.')

['나/NP',
 '는/JX',
 '학교/NNG',
 '에/JKB',
 '갔/VV+EP',
 '습니다/EF',
 './SF',
 '나/NP-는/JX',
 '는/JX-학교/NNG',
 '학교/NNG-에/JKB',
 '갔/VV+EP-습니다/EF',
 '습니다/EF-./SF']

### add new column for title tokenizing

In [110]:
train.head(3)

,index,date,category,media,title,contents,link,title_token
0,0,20170108,IT_science,데일리안,‘효자폰’ 갤럭시S7 ‘불효폰’ G5…분위기 극과극,삼성전자 ‘갤럭시S7’ 왼쪽 LG전자 ‘G5’ 오른쪽 모델 이미지 ⓒ각 사 삼성전자...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[‘, 효자, 폰, ’, 갤럭시, S, 7, ‘, 폰, ’, G, 5, …, 분위기..."
1,1,20170111,IT_science,디지털타임스,카카오 O2O 플랫폼 사업 윤곽도 못잡아,11월초 공개 O2O 위드 카카오 두달 넘게 모습 드러내지 않아 후발주자로 출시까지...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[카카오, O, 2, O, 플랫, 폼, 사업, 윤곽, 도, 못, 잡, 아, O-2,..."
2,2,20170103,economy,조세일보,동원 고위도高緯度 북대서양 참다랑어 어획 성공,동원산업 대표이사 이명우 은 고위도 高緯度 북대서양 참다랑어를 어획하는데 성공했다고...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[동원, 고위, 도, 高, 북대서양, 참다랑어, 어획, 성공, 고위-도, 북대서양-..."


In [111]:
train['title_token'] = train['title'].apply(lambda sent: tokenizer.tokenize(sent))
train.head(3)

,index,date,category,media,title,contents,link,title_token
0,0,20170108,IT_science,데일리안,‘효자폰’ 갤럭시S7 ‘불효폰’ G5…분위기 극과극,삼성전자 ‘갤럭시S7’ 왼쪽 LG전자 ‘G5’ 오른쪽 모델 이미지 ⓒ각 사 삼성전자...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[‘/SY, 효자/NNG, 폰/NNG, ’/SY, 갤럭시/NNP, S/SL, 7/S..."
1,1,20170111,IT_science,디지털타임스,카카오 O2O 플랫폼 사업 윤곽도 못잡아,11월초 공개 O2O 위드 카카오 두달 넘게 모습 드러내지 않아 후발주자로 출시까지...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[카카오/NNP, O/SL, 2/SN, O/SL, 플랫/NNG, 폼/NNG, 사업/..."
2,2,20170103,economy,조세일보,동원 고위도高緯度 북대서양 참다랑어 어획 성공,동원산업 대표이사 이명우 은 고위도 高緯度 북대서양 참다랑어를 어획하는데 성공했다고...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[동원/NNG, 고위/NNG, 도/JX, 高/XPN, 북대서양/NNG, 참다랑어/N..."


In [112]:
print(train['title_token'][0])

['‘/SY', '효자/NNG', '폰/NNG', '’/SY', '갤럭시/NNP', 'S/SL', '7/SN', '‘/SY', '폰/NNG', '’/SY', 'G/SL', '5/SN', '…/SE', '분위기/NNG', '극/NNG', '과/JC', '극/NNG', '폰/NNG-’/SY', '갤럭시/NNP-S/SL', 'S/SL-7/SN', '폰/NNG-’/SY', 'G/SL-5/SN']
